In [10]:
import tensorflow as tf
from tf_agents import policies

saved_policy = tf.compat.v2.saved_model.load("policy_200000")
get_initial_state_fn = saved_policy.signatures['get_initial_state']

In [2]:
get_initial_state_fn

<ConcreteFunction signature_wrapper(*, batch_size) at 0x21B07563100>

In [11]:
from tf_agents.environments.wrappers import ActionRepeat
from gym.wrappers import TimeLimit
from tf_agents.environments.atari_preprocessing import AtariPreprocessing
from tf_agents.environments.atari_wrappers import FrameStack4
from tf_agents.environments import suite_gym
from tf_agents.environments import TFPyEnvironment

class AtariPreprocessingWithSkipStart(AtariPreprocessing):
    def skip_frames(self, num_skip):
        for _ in range(num_skip):
          super().step(0) # NOOP for num_skip steps
    def reset(self, **kwargs):
        obs = super().reset(**kwargs)
        self.skip_frames(40)
        return obs
    def step(self, action):
        lives_before_action = self.ale.lives()
        obs, rewards, done, info = super().step(action)
        if self.ale.lives() < lives_before_action and not done:
            self.skip_frames(40)
        return obs, rewards, done, info


# repeating_env = ActionRepeat(env, times=4)
limited_repeating_env = suite_gym.load(
    "ALE/SpaceInvaders-v5",
    gym_env_wrappers=[lambda env: TimeLimit(env, max_episode_steps=10000)],
    env_wrappers=[lambda env: ActionRepeat(env, times=4)]
)
env = suite_gym.load(
    'ALE/SpaceInvaders-v5',
    max_episode_steps=27000,
    gym_env_wrappers=[AtariPreprocessingWithSkipStart, FrameStack4]
)
tf_env = TFPyEnvironment(env)

c:\Users\clare\anaconda3\envs\late-tf-env\lib\site-packages\gym\utils\seeding.py:138: DeprecationWarning: WARN: Function `hash_seed(seed, max_bytes)` is marked as deprecated and will be removed in the future. 
  deprecation(
c:\Users\clare\anaconda3\envs\late-tf-env\lib\site-packages\gym\utils\seeding.py:175: DeprecationWarning: WARN: Function `_bigint_from_bytes(bytes)` is marked as deprecated and will be removed in the future. 
  deprecation(


In [12]:
from matplotlib import animation
import matplotlib.pyplot as plt
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver



frames = []

class ShowProgress():
    def __init__(self, total):
        self.counter = 0
        self.total = total
    def __call__(self, trajectory):
        if not trajectory.is_boundary():
            self.counter += 1
        if self.counter % 100 == 0:
            log_metrics(train_metrics)
            print("\r{}/{}".format(self.counter, self.total), end="")


In [13]:
from tf_agents.metrics import tf_metrics

train_metrics = [
    tf_metrics.NumberOfEpisodes(),
    tf_metrics.EnvironmentSteps(),
    tf_metrics.AverageReturnMetric(),
    tf_metrics.AverageEpisodeLengthMetric()
]

In [14]:
def update_scene(num, frames, patch):
    patch.set_data(frames[num])
    return patch,

def plot_animation(frames, repeat=False, interval=40):
    fig = plt.figure()
    patch = plt.imshow(frames[0])
    plt.axis('off')
    anim = animation.FuncAnimation(
        fig, update_scene, fargs=(frames, patch),
        frames=len(frames), repeat=repeat, interval=interval)
    plt.close()
    return anim

def save_frames(trajectory):
    global frames
    frames.append(tf_env.pyenv.envs[0].render(mode="rgb_array"))

watch_driver = DynamicStepDriver(
    tf_env,
    saved_policy,
    observers=[save_frames, ShowProgress(5000)] + train_metrics,
    num_steps=5000)
final_time_step, final_policy_state = watch_driver.run()

INFO:absl: 
		 NumberOfEpisodes = 0
		 EnvironmentSteps = 99
		 AverageReturn = 0.0
		 AverageEpisodeLength = 0.0


100/5000

INFO:absl: 
		 NumberOfEpisodes = 1
		 EnvironmentSteps = 199
		 AverageReturn = 465.0
		 AverageEpisodeLength = 139.0


200/5000

INFO:absl: 
		 NumberOfEpisodes = 2
		 EnvironmentSteps = 299
		 AverageReturn = 497.5
		 AverageEpisodeLength = 121.5


...

4800/5000

INFO:absl: 
		 NumberOfEpisodes = 48
		 EnvironmentSteps = 4899
		 AverageReturn = 380.5
		 AverageEpisodeLength = 95.30000305175781


4900/5000

INFO:absl: 
		 NumberOfEpisodes = 50
		 EnvironmentSteps = 4999
		 AverageReturn = 352.0
		 AverageEpisodeLength = 96.5999984741211


5000/5000

In [7]:
from tf_agents.eval.metric_utils import log_metrics
import logging

logging.getLogger().setLevel(logging.INFO)
log_metrics(train_metrics)

INFO:absl: 
		 NumberOfEpisodes = 0
		 EnvironmentSteps = 99
		 AverageReturn = 0.0
		 AverageEpisodeLength = 0.0


In [9]:
import PIL
import os

frame_images = [PIL.Image.fromarray(frame) for frame in frames[1000:]]
image_path = os.path.join("pictures", "invader_test_400000_1.gif")
frame_images[0].save(image_path,format='GIF',
                     append_images=frame_images[1:],
                     save_all=True,
                     duration=60,
                     loop=0)